## The first big project - Professionally You!

### And, Tool use.

### But first: introducing Pushover

Pushover is a nifty tool for sending Push Notifications to your phone.

It's super easy to set up and install!

Simply visit https://pushover.net/ and click 'Login or Signup' on the top right to sign up for a free account, and create your API keys.

Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name (like Agents) and click Create Application.

Then add 2 lines to your `.env` file:

PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
PUSHOVER_TOKEN=_put the key when you click into your new application called Agents (or whatever) and probably starts with an a_

Finally, click "Add Phone, Tablet or Desktop" to install on your phone.

In [ ]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# The usual start

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# For pushover

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
import datetime

# Get the current date and time as a datetime object
now = datetime.datetime.now()

# You can format it into any string representation you like including AM/PM
formatted_date = now.strftime("%a %b %d %I:%M:%S %p %Z %Y")

push("Starting up...  Current time: " + formatted_date)

In [ ]:
push("Starting up...  Current time: " + str(os.popen("date").read().strip()))

In [ ]:
push("HEY!!")

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording \"{question}\" asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

# def handle_tool_calls(tool_calls):
#     results = []
#     for tool_call in tool_calls:
#         tool_name = tool_call.function.name
#         arguments = json.loads(tool_call.function.arguments)
#         print(f"Tool called: {tool_name}", flush=True)

#         # THE BIG IF STATEMENT!!!

#         if tool_name == "record_user_details":
#             result = record_user_details(**arguments)
#         elif tool_name == "record_unknown_question":
#             result = record_unknown_question(**arguments)

#         results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
#     return results

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

In [ ]:
# A more robust and secure way is to use an explicit "dispatch table" or "tool registry".
# This avoids using globals() and makes it clear which functions are available to the agent.

# 1. Create the tool registry dictionary, mapping tool names to functions.
tool_registry = {
    "record_user_details": record_user_details,
    "record_unknown_question": record_unknown_question
}

# 2. Update the handler to use this registry.
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        
        # Look up the function in our secure registry
        tool_function = tool_registry.get(tool_name)
        
        if tool_function:
            # Call the function if it's found
            result = tool_function(**arguments)
        else:
            # Handle cases where the LLM might call a non-existent tool
            result = {"error": f"Tool '{tool_name}' not found."}
            
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
reader = PdfReader("me/untitled_folder/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/untitled_folder/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

# name = "Ed Donner"
name = "Oliver Dreger"  

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile and from {name}'s resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
print(system_prompt  )

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## And now for deployment

This code is in `app.py`

We will deploy to HuggingFace Spaces. Thank you student Robert M for improving these instructions.

Before you start: remember to update the files in the "me" directory - your LinkedIn profile and summary.txt - so that it talks about you!  
Also check that there's no README file within the 1_foundations directory. If there is one, please delete it. The deploy process creates a new README file in this directory for you.

1. Visit https://huggingface.co and set up an account  
2. From the Avatar menu on the top right, choose Access Tokens. Choose "Create New Token". Give it WRITE permissions.
3. Take this token and add it to your .env file: `HF_TOKEN=hf_xxx` and see note below if this token doesn't seem to get picked up during deployment  
4. From the 1_foundations folder, enter: `uv run gradio deploy` and if for some reason this still wants you to enter your HF token, then interrupt it with ctrl+c and run this instead: `uv run dotenv -f ../.env run -- uv run gradio deploy` which forces your keys to all be set as environment variables   
5. Follow its instructions: name it "career_conversation", specify app.py, choose cpu-basic as the hardware, say Yes to needing to supply secrets, provide your openai api key, your pushover user and token, and say "no" to github actions.  

#### Extra note about the HuggingFace token

A couple of students have mentioned the HuggingFace doesn't detect their token, even though it's in the .env file. Here are things to try:   
1. Restart Cursor   
2. Rerun load_dotenv(override=True) and use a new terminal (the + button on the top right of the Terminal)   
3. In the Terminal, run this before the gradio deploy: `$env:HF_TOKEN = "hf_XXXX"`  
Thank you James and Martins for these tips.  

#### More about these secrets:

If you're confused by what's going on with these secrets: it just wants you to enter the key name and value for each of your secrets -- so you would enter:  
`OPENAI_API_KEY`  
Followed by:  
`sk-proj-...`  

And if you don't want to set secrets this way, or something goes wrong with it, it's no problem - you can change your secrets later:  
1. Log in to HuggingFace website  
2. Go to your profile screen via the Avatar menu on the top right  
3. Select the Space you deployed  
4. Click on the Settings wheel on the top right  
5. You can scroll down to change your secrets, delete the space, etc.

#### And now you should be deployed!

Here is mine: https://huggingface.co/spaces/ed-donner/Career_Conversation

I just got a push notification that a student asked me how they can become President of their country 😂😂

For more information on deployment:

https://www.gradio.app/guides/sharing-your-app#hosting-on-hf-spaces

To delete your Space in the future:  
1. Log in to HuggingFace
2. From the Avatar menu, select your profile
3. Click on the Space itself and select the settings wheel on the top right
4. Scroll to the Delete section at the bottom
5. ALSO: delete the README file that Gradio may have created inside this 1_foundations folder (otherwise it won't ask you the questions the next time you do a gradio deploy)


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• First and foremost, deploy this for yourself! It's a real, valuable tool - the future resume..<br/>
            • Next, improve the resources - add better context about yourself. If you know RAG, then add a knowledge base about you.<br/>
            • Add in more tools! You could have a SQL database with common Q&A that the LLM could read and write from?<br/>
            • Bring in the Evaluator from the last lab, and add other Agentic patterns.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">Aside from the obvious (your career alter-ego) this has business applications in any situation where you need an AI assistant with domain expertise and an ability to interact with the real world.
            </span>
        </td>
    </tr>
</table>

## Transcript Summary

### Introduction and Project Overview

*   **Context:** The transcript marks the beginning of the final day (Day 5) of the first week of the course. It's time to start the first major project.
*   **Project Name & Goal:** The project is called "Professionally You" and aims to create a personal career "alter ego" for a website. This AI chatbot will answer questions about the user's professional history.
*   **Technical Approach:** The project is being built by interacting *directly* with AI models, intentionally avoiding any "agentic frameworks." This is a foundational exercise to provide a deep understanding of what happens "under the hood" before using frameworks that abstract these details away.
*   **Key Feature:** The project will implement **tool use**, a concept previously covered in a lecture.

### Introducing the "Pushover" Tool

*   **Purpose:** Pushover is a simple tool for sending push notifications to a phone. It's presented as a superior alternative to services like Twilio for sending SMS, as Twilio has become difficult to use due to heavy regulation.
*   **Setup Process:**
    1.  Create an account on the Pushover website.
    2.  Obtain two keys: a **user key** and an **API token**.
    3.  Store these keys in an environment file (e.g., `.env`) to be loaded by the application.
    4.  Install the Pushover app on your phone.
*   **Cost:** It is free for the first month, after which it costs a "tiny amount."
*   **Implementation and Demonstration:**
    *   A Python function named `push` is created. It makes a `requests.post` call to the Pushover API endpoint.
    *   The data sent in the request includes the user key, API token, and the message text.
    *   A live demonstration is performed, successfully sending a "Hey" message as a push notification to the speaker's phone, which is heard audibly.
*   **Future Use:** This tool is noted as being useful for future projects as well.

### Creating Functions to be Used as Tools

Two Python functions are defined to be equipped as tools for the LLM:

1.  **`record_user_details`:** This function is intended to be used when a website visitor wants to get in touch. It takes an email, name, and notes as input and sends a push notification with this information.
2.  **`record_unknown_question`:** This function is for when the LLM is asked a question it doesn't know the answer to. It takes the question as input and sends a push notification containing that question.

### Converting Functions into Tools using JSON

*   **Core Concept:** The speaker reiterates that "tool use is just JSON and if statements." The process involves manually creating verbose JSON descriptions for each function, a task that agentic frameworks typically automate.
*   **JSON Schema:** For each function, a detailed JSON object is created. This "blob of JSON" describes the tool to the LLM and includes:
    *   **`name`:** The string name of the function (e.g., `"record_user_details"`).
    *   **`description`:** A natural language explanation of the tool's purpose, which the LLM uses to decide when to use it.
    *   **`parameters`:** An object defining the function's arguments, including their `type`, `description`, and which ones are `required`.
*   **Final `tools` List:** Both JSON objects are then placed into a final list called `tools`. Each item in this list is a dictionary specifying the `type` as `"function"` and the `function` as the corresponding JSON blob.
*   **Why JSON?** This specific structure is used because it's a format that LLMs understand well due to its prevalence in their training data, facilitating effective interaction with the OpenAI API. The speaker emphasizes the importance of running all code cells to avoid `NameError` later.

--- 

### The Purpose and Core Logic of `handle_tool_calls`

*   **Introduction:** This section is described as the most important and complex part, explaining how to handle the LLM's response when it decides to use a tool.
*   **The Function's Role:** A function named `handle_tool_calls` is created to process the LLM's output. Its specific purpose is to execute the tool(s) the LLM has requested and then return the results of those tool executions.
*   **Input and Structure:**
    *   The function takes a list of `tool_calls` as input. The `tool_calls` object is structured output from the LLM (JSON parsed into a Python object), making tool use analogous to structured output generation.
    *   It loops through this list. Although a model typically calls only one tool at a time, the API supports multiple concurrent tool calls, so the code is written to handle this.
*   **Process Inside the Loop:**
    1.  It extracts the `function.name` (the string name of the tool to be called).
    2.  It extracts the `function.arguments` (the parameters for that tool).

### Implementation 1: The `if` Statement Approach

*   **Core Logic:** The initial implementation uses a simple and explicit `if` statement to route the tool call.
    *   `if tool_name == "record_user_details":` then the `record_user_details()` function is called with the provided arguments.
    *   `elif tool_name == "record_unknown_question":` then the `record_unknown_question()` function is called.
*   **Result Handling:** The value returned from the executed function is then appended to a list of results, which is ultimately returned by `handle_tool_calls`.
*   **Speaker's Point:** This concrete example demonstrates the speaker's earlier promise that tool use is fundamentally just "if statements."

### Implementation 2: The "Sneaky" and Dynamic Python Approach

*   **The Problem with the `if` statement:** The speaker points out that checking for a string and then calling a function with the exact same name seems redundant and "a bit dumb."
*   **The "Magic" Solution:** Python's built-in `globals()` function is introduced.
    *   `globals()` returns a dictionary of all objects in the global scope, including defined functions.
    *   This allows you to use a string (the tool name) as a key to look up and retrieve the actual function object itself (e.g., `globals().get("record_unknown_question")`).
*   **Live Demonstration:** The speaker proves this trick works by successfully calling a function using the `globals()` lookup and triggering a push notification on their phone.
*   **Refactored Function:** The `handle_tool_calls` function is rewritten to use this dynamic lookup, which eliminates the explicit `if/elif` block.

### The Core Takeaway and Future Context

*   **It's Still a "Glorified If Statement":** The speaker strongly emphasizes that even though the `globals()` trick is clever, it should not be mistaken for something more than it is. The fundamental logic remains the same: mapping a text string to a specific function call.
*   **Why Learn This Manual Process:**
    *   This is a one-time learning exercise for week one. The audience is reassured they will **never have to do this again**.
    *   The purpose is to provide a deep, foundational understanding of what is happening "under the hood."
*   **The Role of Frameworks:** This manual work of parsing JSON and routing it to function calls is **exactly the problem that agentic frameworks solve**. Frameworks were created by developers who wrote code like this and decided to package it to make it easier for everyone else.
*   **Final Goal:** By understanding this foundational layer, users will know precisely what frameworks are doing for them when they start using them in subsequent weeks.

--- 

### Introduction and Project Overview

*   **Context:** This is the final day (Day 5) of the first week of a course. It marks the start of the first major project.
*   **Project Name:** "Professionally You"
*   **Project Goal:** To build a personal career "alter ego" for a website. This AI will be a chatbot that answers questions about the user's professional history.
*   **Technical Approach:** The project will be built by interacting *directly* with AI models without using any "agentic frameworks." This is intentional, as it provides foundational knowledge of what happens "under the hood" before abstracting these details away with frameworks in later lessons.
*   **Key Feature:** The project will incorporate **tool use**, a concept covered in the previous lecture.

### Introducing the "Pushover" Tool

*   **Purpose:** Pushover is a simple tool for sending push notifications to a phone. It's introduced as a practical alternative to services like Twilio for sending text messages, which have become complex due to SMS regulations.
*   **Setup:**
    1.  Visit the Pushover website and set up an account.
    2.  You will receive two keys: a **user key** and an **API token**. These need to be stored in an environment file (like `.env`).
    3.  Install the Pushover app on your phone.
*   **Cost:** Pushover is free for the first month, after which there is a small fee.
*   **Implementation:**
    *   The speaker demonstrates a Python function named `push` that makes a `requests.post` call to the Pushover API endpoint.
    *   The data sent in the request includes the user key, the API token, and the message text.
    *   A live demonstration shows a push notification successfully appearing on the speaker's phone with an audible alert.
*   **Importance:** This tool will be used in this project and future projects as a useful way to receive code-driven notifications.

### Creating Functions to be Used as Tools

Two Python functions are introduced, which will be equipped as tools for the AI to use:

1.  **`record_user_details(email, name, notes)`:**
    *   **Purpose:** To be called when a user expresses interest in getting in touch and provides their contact information.
    *   **Action:** It uses the `push` function to send a notification to the speaker's phone with the user's details.
    *   **Returns:** A dictionary `{"recorded": "ok"}`.

2.  **`record_unknown_question(question)`:**
    *   **Purpose:** To be called when a user asks a question that the AI does not know how to answer.
    *   **Action:** It uses the `push` function to send a notification containing the question that couldn't be answered.

### Converting Functions into Tools using JSON

*   **Core Concept:** The speaker reiterates that "tool use is just JSON and if statements." The process involves describing the Python functions in a specific JSON format that the LLM can understand. While agentic frameworks automate this, it's being done manually here for learning purposes.
*   **JSON Schema:** For each function, a detailed JSON object (a "blob of JSON") is created with the following key properties:
    *   **`name`:** The name of the function (e.g., `"record_user_details"`).
    *   **`description`:** A clear, natural language explanation of what the tool does and when to use it. The LLM uses this to decide if calling the tool is appropriate.
    *   **`parameters`:** An object describing the function's arguments.
        *   **`properties`:** Defines each parameter (e.g., `email`, `name`, `notes`), its `type` (e.g., "string"), and a `description`.
        *   **`required`:** An array listing which parameters are mandatory (e.g., `["email"]`).
*   **Final Structure:** Both of these JSON "blobs" are then combined into a main list called `tools`. Each item in the list is a dictionary specifying the `type` ("function") and the `function` (the JSON blob itself).
*   **Why JSON:** This JSON structure is a "language" that LLMs are very good at understanding because it was a significant part of their training data, enabling effective interaction with the OpenAI API. The speaker runs all the code cells to ensure the `tools` variable is properly defined for later use.